<a href="https://colab.research.google.com/github/tima0476/hw1-csci5922/blob/master/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Neural Networks and Deep Learning (CSCI 5922)</center>
# <center> Spring 2020 </center>

**Name:** Timothy Mason

## Goal

The goal of this assignment is to introduce neural networks in terms of ideas you are already familiar with:  linear regression and classification

## Dataset

You are given a dataset with 2 input variables ($x_1$, $x_2$) and an output variable ($y$).

In [1]:
from sklearn.datasets import make_regression
from matplotlib import pyplot
import numpy as np
import os

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Load data from gDrive
    data = np.loadtxt(os.path.join('/content/drive/My Drive/spring2020/hw1-csci5922/data', 'assign1_data.txt'),  delimiter=',')
except:
    # Load data - relative path from notebook
    data = np.loadtxt(os.path.join('data', 'assign1_data.txt'),  delimiter=',')
    
X = data[:,:2]
y = data[:, 2]
z = data[:, 3]

Mounted at /content/drive


## Part 1

Write a program to find the exact least squares solution to $y = w_1 x_1 + w_2 x_2 + b$ for the above dataset, using the normal equation.

Complete the following function below and use it to answer questions (A) and (B). 

**Note:** Please do not change the interface of the given function.

In [0]:
def least_squares(X, y):
    """
    Finds the Least Squares solution
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of output value 'y' (size : no of examples X 1)
    
    Returns
    -------
    w : solution array
    """

    # From the text:  "Instead of adding the bias parameter b, one can continue to use 
    # the model with only weights but augment x with an extra entry that is always set 
    # to 1. The weight corresponding to the extra 1 entry plays the role of the bias 
    # parameter."

    X = np.c_[np.ones((X.shape[0],1)), X]   # Augment X's with extra 1's

    # Calculate the normal equation.
    w = np.linalg.inv(X.T @ X) @ X.T @ y

    return w

(A) Report the values of $w_1$, $w_2$, and $b$.

In [3]:
w = least_squares(X, y)
print(f"w_1 = {w[1]}\nw_2 = {w[2]}\n  b = {w[0]}")

w_1 = -2.0442425951376353
w_2 = 3.996860168659322
  b = -0.9242908118675891


(B) What function or method did you use to find the least-squares solution?

> The linear regression was calculated using the equation: 
> 
> $$\mathbf{w} = \left( \mathbf{X}^\top \mathbf{X} \right)^{-1} \mathbf{X}^\top \mathbf{y}$$
> 
> 
> The vector $\mathbf{w}$ provides the weight coefficients for the prediction $\hat{y}$:
> 
> $$ \hat{y} = \mathbf{w}^\top \mathbf{x} = w_1 x_1 + w_2 x_2 + \ldots + w_m x_m$$
> 
> 
> Note that the requirement was to find the exact least squares solution to $y = w_1 x_1 + w_2 x_2 + b$.  
>
> To find the $b$ term, an extra column of 1's was inserted as the leftmost column of the $\mathbf{X}$ matrix,
> which adds a new term $x_0 = 1$.  This caused the normal 
> equation to solve for 
> 
> $$
\begin{split}
    \hat{y} = \mathbf{w}^\top \mathbf{x} & =  w_0 x_0 + w_1 x_1 + w_2 x_2 \\
    \\
    \text{Note that we set } x_0 = 1 \text{, therefore: } \hat{y} & = w_0 + w_1 x_1 + w_2 x_2 \\
    \\
    \text{Let }w_0=b\text{, and we have: } \hat{y} & = w_1 x_1 + w_2 x_2 + b
\end{split}
$$
>

## Part 2

Implement linear regression of y on X via first-order optimization of the least-squares objective. Write a program that determines the coefficients {w1,w2,b}. Implement stochastic gradient descent, batch gradient descent, and mini-batch gradient descent. You will need to experiment with updating rules, step sizes (i.e. learning rates), stopping criteria, etc. Experiment to find settings that lead to solutions with the fewest number of sweeps through the data.

Complete the following functions below and use them to answer questions (A), (B) and (C). You may find the shuffle function from scikit-learn useful. 

Use the following hyperparameters:

Learning rates = [0.001, 0.05, 0.01, 0.05, 0.1, 0.3]

MaxIter = [10, 50, 100, 500, 1000, 5000, 10000, 25000, 50000]

**Note:** Please do not change the interface of the given functions.

In [0]:
from sklearn.utils import shuffle

def online_epoch(X, y, w, alpha):
    """
    One epoch of stochastic gradient descent (i.e. one sweep of the dataset).
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    alpha : learning rate
    
    Returns
    -------
    a tuple containing Coefficients of the model (after updating) and the loss
    """
    # Iterate through the training set, updating the model after each example is evaluated
    N = X.shape[0]
    for i in range(N):
        x = X[i]
        # make a prediction with our current set of weights (current example)
        y_hat = x @ w

        # compute the error (current example)
        err = y[i] - y_hat

        # compute the gradient (current example)
        grad = -(x.T * err)

        # update the model after learning from each example
        w -= alpha*grad

    # Return the updated weights and the full-training set loss.
    return (w,np.average((y - X @ w) ** 2))


def batch_update(X, y, w, alpha):
    """
    One iteration of full-batch gradient descent.
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    alpha : Learning rate
    
    Returns
    -------
    a tuple containing Coefficients of the model (after updating) and the loss
    """
    # make a prediction with our current set of weights (full training set)
    y_hat = X @ w

    # compute the error (full training set)
    err = y - y_hat

    # compute the gradient (full training set)
    N = X.shape[0]
    grad = (X.T @ err) * -2.0 / N

    # Now we've evaluated the entire training set, update the model
    w -= alpha*grad

    # Return the updated weights and the loss.
    return (w,np.average(err**2))

def mini_batch_update(X, y, w, alpha, batch_size):
    """
    One epoch of mini-batch SGD over the entire dataset (i.e. one sweep of the dataset).
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    alpha : learining rate
    batch_size : size of the batch for gradient update
    
    Returns
    -------
    a tuple containing Coefficients of the model (after updating) and the loss
    """
    # Pull out a random subset of the training data.

    mini_comb = shuffle(np.c_[X,y], n_samples = batch_size)     # shuffle combined X and y
    X_sub = mini_comb[:,:3]
    y_sub = mini_comb[:,3]

    # make a prediction with our current set of weights (mini-batch training set)
    y_hat = X_sub @ w

    # compute the error (mini-batch training set)
    err = y_sub - y_hat

    # compute the gradient (full training set)
    grad = (X_sub.T @ err) * -2.0 / batch_size

    # Now we've evaluated the mini-batch, update the model
    w -= alpha*grad

    # Return the updated weights and the full-training set loss.
    return (w,np.average((y - X @ w) ** 2))


def least_squares_grad_desc(X, y, w, maxIter, alpha, update, *batch_size):
    """
    Implements least squares with gradient descent.
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    maxIter : Maximum number of iterations allowed
    alpha : Learning rate
    update : update function to utilize (one of online, batch, mini-batch)
    batch_size : number of examples in a batch (only useful when update = mini_batch_update)
    
    Returns
    -------
    a tuple containing Coefficients of the model (after updating) and the loss
    
    Note : *batch_size is an optional argument and only to be used when doing mini-batch Gradient Descent 
    """
    for i in range(maxIter):
        if batch_size:
            w,l = update(X, y, w, alpha, batch_size[0])
        else:
            w,l = update(X, y, w, alpha)

    return (w,l)


(A) Report the values of $w_1$, $w_2$, and $b$. 

In [0]:
rates = [0.001, 0.05, 0.01, 0.05, 0.1, 0.3]
MaxIter = [10, 50, 100, 500, 1000, 5000, 10000, 25000, 50000]

Xaug = np.c_[np.ones((X.shape[0],1)), X]   # Augment X's with extra 1's for the bias term

### Stochastic Gradient Descent

In [0]:
for alpha in rates:
    w = np.random.uniform(0, 1, (Xaug.shape[1]))  # make a random initial guess at the weights
    for mi in MaxIter:
        w,l = least_squares_grad_desc(Xaug, y, w, mi, alpha, online_epoch)
        print(f"  Rate={alpha}, MaxIter:{mi:7,d}; Loss={l:9.6f}:  w1={w[1]:9.6f}, w2={w[2]:9.6f}, b={w[0]:8.5f}")
    print()


### Batch Gradient Descent

In [0]:
for alpha in rates:
    w = np.random.uniform(0, 1, (Xaug.shape[1]))  # make a random initial guess at the weights
    for mi in MaxIter:
        w,l = least_squares_grad_desc(Xaug, y, w, mi, alpha, batch_update)
        print(f"  Rate={alpha}, MaxIter:{mi:7,d}; Loss={l:9.6f}:  w1={w[1]:9.6f}, w2={w[2]:9.6f}, b={w[0]:8.5f}")
    print()

### Mini-Batch Gradient Descent

In [0]:
for alpha in rates:
    w = np.random.uniform(0, 1, (Xaug.shape[1]))  # make a random initial guess at the weights
    for mi in MaxIter:
        w,l = least_squares_grad_desc(Xaug, y, w, mi, alpha, mini_batch_update, 32)
        print(f"  Rate={alpha}, MaxIter:{mi:7,d}; Loss={l:9.6f}:  w1={w[1]:9.6f}, w2={w[2]:9.6f}, b={w[0]:8.5f}")
    print()

(B) What settings worked well for you:  online vs. batch vs. minibatch? What step size? How did you decide to terminate?

(C) Make a graph of error on the entire data set as a function of epoch. An epoch is a complete sweep through all the data (which is one iteration for full-batch gradient descent).

## Part 3

The data set from a regression problem can be converted into a classification problem simply by using the sign of (+ or -) as representing one of two classes. In the data set used in Part 1 and 2, you'll see the variable z that represents this binary (0 or 1) class.

Use the perceptron learning rule to solve for the coefficients {$w_1$, $w_2$, $b$} of this classification problem.   

Two warnings: First, your solution to Part 3 should require only a few lines of code changed from the code you wrote for Part 2. Second, the Perceptron algorithm will not converge if there is no exact solution to the training data. It will jitter among coefficients that all yield roughly equally good solutions.

Complete the following functions below and use them to answer questions (A) and (B). 

**Note:** Please do not change the interface of the given functions.

In [0]:
def perceptron_update(X, y, w):
    """
    One epoch of Perceptron updates (full sweep of the dataset).
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    w : array of coefficients from the previous iteration
    
    Returns
    -------
    w : Coefficients of the classifier (after updating)
    incorrect : Incorrectly classified examples
    """
    pass

def perceptron(X, y, maxIter, alpha):
    """
    Implements the Perceptron algorithm.
    
    Parameters
    ----------
    X : NumPy array of features (size : no of examples X features)
    y : Numpy array of class labels (size : no of examples X 1)
    maxIter : The maximum number of iterations allowed 
    alpha : Learning Rate
    
    Returns
    -------
    w : Coefficients of the classifier
    incorrect : Incorrectly classified examples on termination
    """
    pass

(A) Report the values of coefficients $w_1$, $w_2$, and $b$.

(B) Make a graph of the accuracy (% correct classification) on the training set as a function of epoch.

## Part 4

In machine learning, we really want to train a model based on some data and then expect the model to do well on "out of sample" data. Try this with the code you wrote for Part 3:  Train the model on the first {5, 10, 25, 50, 75} examples in the data set and test the model on the final 25 examples.

Complete the following function below and use it to answer (A). 

**Note:** Please do not change the interface of the given function.

In [0]:
def classify(X, y, w):
    """
    Use this function to classify examples in the test set
    
    Parameters
    ----------
    X : Test set features
    y : Test set labels
    w : Perceptron coefficients
    
    Returns
    -------
    correct : number of correctly classified examples
    """
    pass

How does performance on the test set vary with the amount of training data? Make a bar graph showing performance for each of the different training set sizes.